
This notebook demonstrates the theory and practice of common image processing techniques using OpenCV, NumPy, and PIL.
It covers:
- Image fundamentals
- Color conversions
- Transformations
- Filtering & Morphology
- Enhancement
- Edge Detection
- Compression


1.import libraries :
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Helper function to display images
def show_image(img, title="Image"):
    if len(img.shape) == 2:  # grayscale
        plt.imshow(img, cmap="gray")
    else:  # color
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis("off")
    plt.show()

# Manual color conversions
def rgb_to_gray_manual(img):
    return np.dot(img[...,:3], [0.299, 0.587, 0.114]).astype(np.uint8)

def rgb_to_hsv_manual(img):
    img = img.astype("float32") / 255.0
    r, g, b = img[...,0], img[...,1], img[...,2]
    maxc = np.max(img, axis=-1)
    minc = np.min(img, axis=-1)
    delta = maxc - minc
    hue = np.zeros_like(maxc)
    mask = delta != 0
    hue[mask & (maxc==r)] = (60*((g[mask]-b[mask])/delta[mask])+360)%360
    hue[mask & (maxc==g)] = (60*((b[mask]-r[mask])/delta[mask])+120)%360
    hue[mask & (maxc==b)] = (60*((r[mask]-g[mask])/delta[mask])+240)%360
    sat = np.zeros_like(maxc)
    sat[maxc!=0] = (delta[maxc!=0]/maxc[maxc!=0])
    val = maxc
    hsv = np.stack([hue/2, sat*255, val*255], axis=-1).astype(np.uint8)
    return hsv

def rgb_to_ycbcr_manual(img):
    mat = np.array([[0.299, 0.587, 0.114],
                    [-0.1687, -0.3313, 0.5],
                    [0.5, -0.4187, -0.0813]])
    shift = np.array([0,128,128])
    ycbcr = img @ mat.T + shift
    return np.clip(ycbcr,0,255).astype(np.uint8)


2.Load and Display Image :

In [ ]:
# Load image
img_path = "sample.jpg"  # replace with your image path
img = cv2.imread(img_path)

# Display original image
show_image(img, "Original Image")


3. Color Conversions:
We demonstrate RGB → Grayscale, HSV, and YCbCr using both OpenCV and manual formulas.

In [ ]:
# OpenCV conversions
gray_cv = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
hsv_cv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
ycbcr_cv = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

# Manual conversions
gray_manual = rgb_to_gray_manual(img)
hsv_manual = rgb_to_hsv_manual(img)
ycbcr_manual = rgb_to_ycbcr_manual(img)

# Display
show_image(gray_cv, "Grayscale (OpenCV)")
show_image(gray_manual, "Grayscale (Manual)")


4.Geometric Transformations :
- Translation (Shift)
- Rotation
- Scaling (Resize)
- Affine Transform
- Perspective Transform

In [ ]:
h, w = img.shape[:2]
M = np.float32([[1, 0, 50], [0, 1, 30]])  # shift x=50, y=30
translated = cv2.warpAffine(img, M, (w,h))
show_image(translated, "Translated Image")

5.Filtering & Morphology :

Demonstrate Gaussian, Median, Mean filters and Morphological operations like Dilation, Erosion, Opening, Closing.


In [ ]:
gaussian = cv2.GaussianBlur(img, (5,5), 0)
show_image(gaussian, "Gaussian Blur")

6.Image Enhancement :
- Histogram Equalization
- Contrast Stretching
- Sharpening

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
equalized = cv2.equalizeHist(gray)
show_image(equalized, "Histogram Equalized")

7.Edge Detection :
- Sobel
- Laplacian
- Canny

In [ ]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
sobel = cv2.convertScaleAbs(sobel)
show_image(sobel, "Sobel Edge Detection")

8.Compression :
Demonstrate saving images in different formats and comparing file sizes.

In [ ]:
from PIL import Image
import io

# Convert to PIL Image
pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

# Save as JPEG
buffer = io.BytesIO()
pil_img.save(buffer, format="JPEG", quality=90)
print(f"JPEG Size: {buffer.getbuffer().nbytes//1024} KB")

# Save as PNG
buffer2 = io.BytesIO()
pil_img.save(buffer2, format="PNG")
print(f"PNG Size: {buffer2.getbuffer().nbytes//1024} KB")